In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

In [2]:
# Load the saved EfficientNetV2 model
model = load_model("efficientnetv2_model.h5")

# Check the model architecture
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 13, 13, 1280)      20331360  
 al)                                                             
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 200)               256200    
                                                                 
Total params: 20587560 (78.54 MB)
Trainable params: 256200 (1000.78 KB)
Non-trainable params: 20331360 (77.56 MB)
_________________________________________________________________


In [3]:
# Freeze the base model's layers (optional, depending on your needs)
for layer in model.layers[:-1]:  # Exclude the output layer
    layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Fine-tuning requires a smaller learning rate
    loss="categorical_crossentropy",     # Ensure the loss matches your use case
    metrics=["accuracy"]
)

In [5]:
# load class_names
labels = np.load("class_names.npy", allow_pickle=True).item()

# load attributes
attributes = np.load("attributes.npy")
print(attributes.shape)
attributes_dict = {i + 1: row for i, row in enumerate(attributes)}
# load training data
train_df = pd.read_csv("train_images.csv")
# add attributes to data
train_df["attributes"] = train_df["label"].map(attributes_dict)

# Initialize empty lists for images (X) and labels (y)
X = []
y = []

# Iterate over rows in the DataFrame
for _, row in train_df.iterrows():
    # Read the image
    im_path = row['image_path']
    image = cv2.imread(f'train_images{im_path}')
    
    # Resize the image to 400x400
    image_resized = cv2.resize(image, (400, 400))
    
    # Append to the list of images and labels
    X.append(image_resized)

# Convert lists to numpy arrays (optional)
X = np.array(X)
y = train_df['label']

(200, 312)


In [6]:
# Convert y to categorical
y_c = to_categorical(y.values-1)
X_train, X_test, y_train, y_test = train_test_split(X, y_c, test_size=0.2, random_state=0)
# use one-hot encoding for labels
print(y_train.shape)

(3140, 200)


In [16]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    class_weight = 'balanced', 
    classes = np.unique(y), 
    y = y
)

# Convert to a dictionary for Keras
class_weight_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weight_dict)

Class Weights: {0: 0.5608571428571428, 1: 0.5608571428571428, 2: 0.5948484848484848, 3: 0.5608571428571428, 4: 1.033157894736842, 5: 1.226875, 6: 0.7010714285714286, 7: 0.8534782608695652, 8: 0.5773529411764706, 9: 0.5773529411764706, 10: 0.5948484848484848, 11: 0.6332258064516129, 12: 0.5948484848484848, 13: 0.5948484848484848, 14: 0.5948484848484848, 15: 0.5948484848484848, 16: 0.6134375, 17: 0.9815, 18: 0.6134375, 19: 0.6134375, 20: 0.6134375, 21: 0.6332258064516129, 22: 0.6134375, 23: 0.727037037037037, 24: 0.6332258064516129, 25: 0.6332258064516129, 26: 0.6332258064516129, 27: 0.6332258064516129, 28: 0.6332258064516129, 29: 0.6332258064516129, 30: 0.6543333333333333, 31: 0.7010714285714286, 32: 0.6543333333333333, 33: 0.6543333333333333, 34: 0.6543333333333333, 35: 0.6543333333333333, 36: 0.6543333333333333, 37: 0.676896551724138, 38: 0.676896551724138, 39: 0.676896551724138, 40: 0.676896551724138, 41: 0.676896551724138, 42: 0.676896551724138, 43: 0.676896551724138, 44: 0.70107142

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Fine-tune the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=5,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

Epoch 1/5
99/99 [==============================] - 776s 8s/step - loss: 0.8150 - accuracy: 0.8952 - val_loss: 1.6316 - val_accuracy: 0.5954
Epoch 2/5
99/99 [==============================] - 503s 5s/step - loss: 0.7981 - accuracy: 0.8949 - val_loss: 1.6290 - val_accuracy: 0.5891
Epoch 3/5
99/99 [==============================] - 512s 5s/step - loss: 0.7784 - accuracy: 0.8971 - val_loss: 1.6291 - val_accuracy: 0.5878
Epoch 4/5
99/99 [==============================] - 518s 5s/step - loss: 0.7686 - accuracy: 0.8968 - val_loss: 1.6274 - val_accuracy: 0.5878
Epoch 5/5
99/99 [==============================] - 506s 5s/step - loss: 0.7527 - accuracy: 0.9096 - val_loss: 1.6232 - val_accuracy: 0.5840


In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

25/25 [==============================] - 97s 4s/step - loss: 1.6301 - accuracy: 0.5980
Test Loss: 1.6301114559173584, Test Accuracy: 0.5979644060134888


In [18]:
test_df = pd.read_csv("test_images_path.csv")

test_images = []
# Iterate over rows in the DataFrame
for _, row in test_df.iterrows():
    # Read the image
    im_path = row['image_path']
    image = cv2.imread(f'test_images{im_path}')
    
    # Resize the image to 400x400
    image_resized = cv2.resize(image, (400, 400))
    
    # Append to the list of images and labels
    test_images.append(image_resized)


predictions = model.predict(np.array(test_images))
predicted_labels = np.argmax(predictions, axis = 1)

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels + 1
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 646s 5s/step


In [23]:
print(min(predicted_labels))

0
